In [1]:
from seqnado.config.third_party_tools import ThirdPartyToolsConfig
from seqnado import Assay
tools_config = ThirdPartyToolsConfig.for_assay(Assay.ATAC)
dummped = tools_config.model_dump(exclude_none=True)

In [3]:
cfg = ThirdPartyToolsConfig(**dummped)  # This line is just to show how to instantiate with the dumped data

In [4]:
cfg.bamnado.bam_coverage

ToolConfig(threads=8, options=OptionsBase(value=' --bin-size 10 --norm-method rpkm', exclude=set()))

In [3]:
from pydantic import BaseModel, Field
from seqnado import Assay
from seqnado.config import SeqnadoConfig, ProjectConfig, PCRDuplicatesConfig, GenomeConfig, BowtieIndex

In [6]:
x = SeqnadoConfig(
    assay=Assay.ATAC,
    genome=GenomeConfig(
        name="hg38",
        index=BowtieIndex(prefix="/Users/asmith/Documents/reference/hg38_ecoli_b2t_index/bt2_index/hg38_ecoli")
    ),
    project=ProjectConfig(name="my_project", date="2025-01-01"),
    metadata='metadata.csv',
    pcr_duplicates=PCRDuplicatesConfig()
).model_dump_json(exclude_none=True)


with open("test.json", "w") as f:
    f.write(x)

In [5]:
wc = WorkflowConfig(
    assay=Assay.CHIP,
    genome=GenomeConfig(
        name="GRCh38",
        index=BowtieIndex(
            prefix="../../../reference/hg38_ecoli_b2t_index/bt2_index/hg38_ecoli"
        ),
    ),
    project=ProjectConfig(
        name="Test Project",
        date=datetime.today(),
        description="This is a test project",
        output_dir="test_output",
        directory="test_directory",
    ),
    metadata=Path("test_metadata.yaml"),
    assay_config=AssayConfig.CHIP.value(
        bigwigs=BigwigConfig(
            pileup_method=[PileupMethod.BAMNADO],
            binsize=100,
        ),
        peak_calling=PeakCallingConfig(
            method=[PeakCallingMethod.MACS2],
            consensus_counts=False,
        ),
    ),
    options=get_tool_options(Assay.RNA),
)


NameError: name 'WorkflowConfig' is not defined

In [ ]:
dumped = wc.model_dump_json()

In [ ]:
WorkflowConfig.model_validate_json(dumped)

WorkflowConfig(assay=<Assay.CHIP: 'ChIP'>, project=ProjectConfig(name='Test Project', date=datetime.datetime(2025, 7, 27, 19, 27, 38, 150976), directory=PosixPath('test_directory')), genome=GenomeConfig(name='GRCh38', index=BowtieIndex(type='Bowtie2', prefix='../../../reference/hg38_ecoli_b2t_index/bt2_index/hg38_ecoli'), fasta=None, chromosome_sizes=None, gtf=None, genes=None, blacklist=None, organism=None, version=None), metadata=PosixPath('test_metadata.yaml'), pcr_duplicates=PCRDuplicatesConfig(strategy=<PCRDuplicateHandling.NONE: 'dont_remove'>, tool=None), qc=QCConfig(calculate_library_size=True, calculate_fraction_of_reads_in_peaks=True), assay_config=ChIPAssayConfig(bigwigs=BigwigConfig(pileup_method=[<PileupMethod.BAMNADO: 'bamnado'>], binsize=100), plotting=None, ucsc_hub=None, dataset_for_ml=None, create_geo_submission_files=False, spikein=None, peak_calling=PeakCallingConfig(method=[<PeakCallingMethod.MACS2: 'macs2'>], consensus_counts=False), create_heatmaps=False, call_pe

In [ ]:
WorkflowConfig.model_fields

{'assay': FieldInfo(annotation=Assay, required=True),
 'project': FieldInfo(annotation=ProjectConfig, required=True),
 'genome': FieldInfo(annotation=GenomeConfig, required=True),
 'metadata': FieldInfo(annotation=Path, required=True),
 'pcr_duplicates': FieldInfo(annotation=PCRDuplicatesConfig, required=False, default=PCRDuplicatesConfig(strategy=<PCRDuplicateHandling.NONE: 'dont_remove'>, tool=None)),
 'qc': FieldInfo(annotation=QCConfig, required=False, default=QCConfig(calculate_library_size=True, calculate_fraction_of_reads_in_peaks=True)),
 'assay_config': FieldInfo(annotation=Union[ATACAssayConfig, ChIPAssayConfig, CATAssayConfig, RNAAssayConfig, SNPAssayConfig, MCCAssayConfig, METHAssayConfig, CRISPRAssayConfig, NoneType], required=False, default=None),
 'options': FieldInfo(annotation=Union[str, NoneType], required=False, default=None)}

In [ ]:
dumped

'{"assay":"ChIP","project":{"name":"Test Project","date":"2025-07-27T19:27:38.150976","directory":"test_directory"},"genome":{"name":"GRCh38","index":{"type":"Bowtie2","prefix":"../../../reference/hg38_ecoli_b2t_index/bt2_index/hg38_ecoli"},"fasta":null,"chromosome_sizes":null,"gtf":null,"genes":null,"blacklist":null,"organism":null,"version":null},"metadata":"test_metadata.yaml","pcr_duplicates":{"strategy":"dont_remove","tool":null},"qc":{"calculate_library_size":true,"calculate_fraction_of_reads_in_peaks":true},"assay_config":{"bigwigs":{"pileup_method":["bamnado"],"binsize":100},"plotting":null,"ucsc_hub":null,"dataset_for_ml":null,"create_geo_submission_files":false,"spikein":null,"peak_calling":{"method":["macs2"],"consensus_counts":false},"create_heatmaps":false,"call_peaks":true,"create_bigwigs":true,"plot_with_plotnado":false,"create_dataset":false,"create_ucsc_hub":false},"options":"bamnado:\\n  bamcoverage: --bin-size 10 --norm-method cpm\\n  threads: 8\\ndeeptools:\\n  alig

In [ ]:
env = Environment(loader=FileSystemLoader("."))
template = env.get_template("template.jinja")

In [ ]:
with open("output.yml", "w") as f:
    r = wc.model_dump(mode="json", exclude_none=True)
    r = template.render(**r)
    f.write(r)


In [ ]:
class ToolConfig(BaseModel):
    """Base model for CLI-based tools."""
    threads: int = Field(default=1, ge=1)
    options: str = Field(default="")


class Bamnado(BaseModel):
    """Configuration for Bamnado tool."""
    bam_coverage: ToolConfig = Field(
        default_factory=lambda: ToolConfig(options="--bin-size 10 --norm-method rpkm ")
    )

class ThirdPartyToolsConfig(BaseModel):
    """Configuration for third-party tools."""
    bamnado: Bamnado = Field(
        default_factory=lambda: Bamnado()
    )

In [ ]:
Bamnado().model_dump_json()

'{"bam_coverage":{"threads":1,"options":"--bin-size 10 --norm-method rpkm "}}'

In [ ]:
ThirdPartyToolsConfig().model_dump_json()

'{"bamnado":{"bam_coverage":{"threads":1,"options":"--bin-size 10 --norm-method rpkm "}}}'